# Day 11: Corporate Policy

[*Advent of Code 2015 day 11*](https://adventofcode.com/2015/day/11) and [*solution megathread*](https://redd.it/3wbzyv)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/11/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F11%2Fcode.ipynb)

In [1]:
%load_ext nb_mypy

Version 1.0.4


In [2]:
%nb_mypy On

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

3:1: E402 module level import not at top of file
6:20: E225 missing whitespace around operator


Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [6]:
from IPython.display import HTML


HTML(downloaded['part1'])

## Comments

It's 2021-12-05, and for some reason I put off solving this one and those after - prefering to fill the repository with more problems. Now it is time though.

_Update 2022-11-19:_ Funny how some problems just feel too tough (and boring?), so I didn't solve this until now... Anyway, the classes (to simplify operations) may be introducing unreasonable costs, so the _test_ problems take frustratingly long to verify.

In [7]:
from IPython.display import display

inputdata = downloaded['input']
display(f'{inputdata =}')

"inputdata ='vzbxkghb'"

In [15]:
from typing import Any, List


class LowerCaseChar:
    def __init__(self, o: int) -> None:
        self.o = o
        self.c = chr(o)

    @staticmethod
    def from_char(c: str):
        return LowerCaseChar(ord(c))

    def __str__(self) -> str:
        return self.c

    def __repr__(self) -> str:
        return f"'{self.c}'"

    def __eq__(self, other: Any) -> bool:
        if isinstance(other, LowerCaseChar):
            return self.o == other.o
        elif isinstance(other, str) and len(other) == 1:
            return self.c == other
        else:
            return NotImplemented

    def increment(self) -> bool:
        """Increment the character and, secondarily, return True to indicate a
        'carry' when incrementing from 'z' to 'a'"""
        self.o += 1
        self.c = chr(self.o)
        if self.c > 'z':
            self.c = 'a'
            self.o = ord(self.c)
            return True
        else:
            return False

In [18]:
from copy import deepcopy


class Password:
    def __init__(self, cs: List[LowerCaseChar]):
        # Note that internally, characters are stored in a list
        # least significant position first
        self.cs = cs

    def __str__(self) -> str:
        return ''.join([str(c) for c in self.cs][::-1])

    def __repr__(self) -> str:
        return f"'{str(self)}'"

    @staticmethod
    def from_string(cs: str):
        return Password([LowerCaseChar.from_char(c) for c in cs][::-1])

    # Passwords must include one increasing straight of at least three letters,
    #   like abc, bcd, cde, and so on, up to xyz. They cannot skip letters; abd
    #   doesn't count.
    def rule_3straight(self) -> bool:
        # Note that we go through characters least significant position
        # first
        triples = zip(deepcopy(self.cs)[:-2],
                      deepcopy(self.cs)[1:-1],
                      deepcopy(self.cs)[2:])
        for triple in triples:
            if (
                    not any([triple[1].increment(),
                             triple[2].increment(),
                             triple[2].increment()])
                    and triple[0] == triple[1]
                    and triple[1] == triple[2]):
                return True
        return False

    # Passwords may not contain the letters i, o, or l, as these letters can
    #   be mistaken for other characters and are therefore confusing.
    # Instantiate static objects, since I suspect this comparison otherwise
    # is unreasonably costly
    def rule_noconfusing(self) -> bool:
        return not any(confusing in str(self) for confusing in 'iol')

    # Passwords must contain at least two different, non-overlapping pairs of
    #   letters, like aa, bb, or zz.
    def rule_twopairs(self) -> bool:
        self_str = str(self)
        bigrams = zip(self_str[:-1], self_str[1:])
        pairs = [False] + \
            list(map(lambda bg: bg[0] == bg[1], bigrams)) + \
            [False]
        overlapping = zip(pairs[:-1], pairs[1:])
        non_overlapping = map(lambda ol: any(ol), overlapping)
        return sum(non_overlapping) >= 4

    def is_valid(self) -> bool:
        # Evaluate these lazily in the order of fastest to most complex
        return all(f() for f in
                   (self.rule_noconfusing,
                    self.rule_3straight,
                    self.rule_twopairs))

    def next(self):
        next_cs = deepcopy(self.cs)
        # Increment starting from least significant until we don't carry
        for c in next_cs:
            if not c.increment():
                break
        return Password(next_cs)

    def next_valid(self):
        next_pass = self.next()
        while not next_pass.is_valid():
            next_pass = next_pass.next()
        return next_pass

In [16]:
hijklmmn = Password.from_string("hijklmmn")
assert hijklmmn.rule_3straight() == True, "hijklmmn meets the first requirement " + \
    "(because it contains the straight hij)"
assert hijklmmn.rule_noconfusing() == False, "...but fails the second requirement " + \
    "requirement (because it contains i and l)"
abbceffg = Password.from_string("abbceffg")
assert abbceffg.rule_twopairs() == True, "abbceffg meets the third requirement " + \
    "(because it repeats bb and ff)"
assert abbceffg.rule_3straight() == False, "...but fails the first requirement"
abbcegjk = Password.from_string("abbcegjk")
assert abbcegjk.rule_twopairs() == False, "abbcegjk fails the third requirement, " + \
    "because it only has one double letter (bb)"
abcdefgh = Password.from_string("abcdefgh")
assert str(abcdefgh.next_valid()) == "abcdffaa", "The next password after " + \
    "abcdefgh is abcdffaa"
ghijklmn = Password.from_string("ghijklmn")
next = ghijklmn.next_valid() 
display(f'{next=}')
assert str(next) == "ghjaabcc", "The next password after " + \
    "ghijklmn is ghjaabcc, because you eventually skip all the passwords that " + \
    "start with ghi..., since i is not allowed"

"next='ghjaabcc'"

ValueError: too many values to unpack (expected 3)

In [ ]:
def my_part1_solution(data: str) -> str:
    return str(Password.from_string(data).next_valid())

In [ ]:
display(my_part1_solution(inputdata))

'vzbxxyzz'

In [ ]:
HTML(downloaded['part1_footer'])

## Part Two

In [ ]:
HTML(downloaded['part2'])

In [ ]:
def my_part2_solution(data: str) -> str:
    return str(Password.from_string(data).next_valid().next_valid())

In [ ]:
display(my_part2_solution(inputdata))

'vzcaabcc'

In [ ]:
HTML(downloaded['part2_footer'])